In [ ]:
import h5py
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import copy
from typing import List, Dict
import pandas as pd
from pydmd import DMD
from pydmd.plotter import plot_summary
from scipy.fft import dct,fft
from scipy.interpolate import CubicSpline, interp1d, Akima1DInterpolator
import itertools
import time
plt.style.use('ggplot')
# plt.rcParams["figure.figsize"] = (12,10)
plt.rcParams["figure.figsize"] = (10,8)

# Functions to read worldtube data

In [ ]:
def generate_columns(num_cols:int,beta_type=False):
  if beta_type:
    num_cols = num_cols*2
  L_max = int(np.sqrt((num_cols-1)/2))-1
  # print(L_max,np.sqrt((num_cols-1)/2)-1)
  col_names = ['t(M)']
  for l in range(0,L_max+1):
    for m in range(-l,l+1):
      if beta_type:
        if m==0:
          col_names.append(f"Re({l},{m})")
        elif m < 0:
          continue
        else:
          col_names.append(f"Re({l},{m})")
          col_names.append(f"Im({l},{m})")
      else:
        col_names.append(f"Re({l},{m})")
        col_names.append(f"Im({l},{m})")
  return col_names


def horizon_to_pandas(horizon_path:Path):
    assert(horizon_path.exists())
    df_dict = {}
    beta_type_list = ['Beta.dat', 'DuR.dat', 'R.dat', 'W.dat']
    with h5py.File(horizon_path,'r') as hf:
        # Not all horizon files may have AhC
        for key in hf.keys():
            if key == "VersionHist.ver":
              continue 
            if key in beta_type_list:
              df_dict[key] = pd.DataFrame(hf[key], columns=generate_columns(hf[key].shape[1],beta_type=True))
            else:
              df_dict[key] = pd.DataFrame(hf[key], columns=generate_columns(hf[key].shape[1]))


    return df_dict


# chebyshev fit

In [ ]:
def cheby_extrema(n:int):
    x = np.linspace(0,n,n+1)*np.pi/n
    return np.cos(x)
def cheby_zeros(n:int):
    x = (np.linspace(0,n-1,n)+0.5)*np.pi/n
    return np.cos(x)

def cheby_eval_recc(n: int, x: np.ndarray) -> np.ndarray:
    """
    Evaluate the nth Chebyshev polynomial of the first kind at points x.

    Args:
    n (int): Degree of the Chebyshev polynomial.
    x (np.ndarray): Points at which to evaluate the polynomial.

    Returns:
    np.ndarray: Values of the Chebyshev polynomial at points x.
    """
    memo = {0: np.ones_like(x), 1: x}

    def cheby_eval_recc_helper(k: int) -> np.ndarray:
        if k not in memo:
            memo[k] = 2 * x * cheby_eval_recc_helper(k - 1) - cheby_eval_recc_helper(k - 2)
        return memo[k]

    return cheby_eval_recc_helper(n)


def evaluate_chebyshevII(c: np.ndarray, x: np.ndarray) -> np.ndarray:
    """
    Evaluate a Chebyshev polynomial fit at points x.

    Args:
    c (np.ndarray): Coefficients of the Chebyshev polynomial fit.
    x (np.ndarray): Points at which to evaluate the polynomial.

    Returns:
    np.ndarray: Values of the Chebyshev polynomial at points x.
    """
    N = len(c)
    x_scaled = 2 * (x - np.min(x)) / (np.max(x) - np.min(x)) - 1
    y = np.zeros_like(x)
    for j in range(N):
        y += c[j] * cheby_eval_recc(j, x_scaled)
    return y

def evaluate_chebyshevII(c: np.ndarray, x: np.ndarray) -> np.ndarray:
    """
    Evaluate a Chebyshev polynomial fit at points x.

    Args:
    c (np.ndarray): Coefficients of the Chebyshev polynomial fit.
    x (np.ndarray): Points at which to evaluate the polynomial.

    Returns:
    np.ndarray: Values of the Chebyshev polynomial at points x.
    """
    N = len(c)
    x_scaled = 2 * (x - np.min(x)) / (np.max(x) - np.min(x)) - 1
    
    # Use Clenshaw's recurrence formula
    b_k1 = np.zeros_like(x_scaled)
    b_k2 = np.zeros_like(x_scaled)
    
    for k in range(N - 1, 0, -1):
        b_k = c[k] + 2 * x_scaled * b_k1 - b_k2
        b_k2 = b_k1
        b_k1 = b_k
    
    return c[0] + x_scaled * b_k1 - b_k2


def fit_chebyshev_dctII(f: callable, N: int, x_min: float, x_max: float) -> np.ndarray:
    """
    Fit a 1D function using Chebyshev polynomials.

    Args:
    f (callable): Function to fit.
    N (int): Number of Chebyshev polynomials to use.
    x_min (float): Minimum x value.
    x_max (float): Maximum x value.

    Returns:
    np.ndarray: Coefficients of the Chebyshev polynomial fit.
    """
    x_scaled = x_min + (x_max - x_min) * (cheby_zeros(N) + 1) / 2
    f_k = f(x_scaled)
    C_j = dct(f_k,type=2,norm='backward')/N
    C_j[0] /= 2
    return C_j

In [ ]:
def plot_interpolation_error(x,y,x_min=None,x_max=None,N_interp = None,plot_axis=None):
    if x_min is None:
      x_min = x.min()
    if x_max is None:
      x_max = x.max()
    if N_interp is None:
      N_interp = len(x)

    # Create a finer x-grid for evaluation
    x_fine = np.linspace(x_min, x_max, num=N_interp)

    # Original CubicSpline interpolation
    cubic_spline = CubicSpline(x, y)
    y_cubic = cubic_spline(x_fine)

    # Linear interpolation
    linear_interp = interp1d(x, y, kind='linear')
    y_linear = linear_interp(x_fine)

    # Quadratic interpolation
    quadratic_interp = interp1d(x, y, kind='quadratic')
    y_quadratic = quadratic_interp(x_fine)

    # Akima interpolation
    akima_interp = Akima1DInterpolator(x, y)
    y_akima = akima_interp(x_fine)

    # Calculate differences
    diff_linear = np.abs(y_cubic - y_linear)
    diff_quadratic = np.abs(y_cubic - y_quadratic)
    diff_akima = np.abs(y_cubic - y_akima)

    # Calculate relative differences
    with np.errstate(divide='ignore', invalid='ignore'): #ignore division by zero warnings
        rel_diff_linear = np.abs((y_cubic - y_linear) / np.where(y_cubic != 0, y_cubic, np.inf))
        rel_diff_quadratic = np.abs((y_cubic - y_quadratic) / np.where(y_cubic != 0, y_cubic, np.inf))
        rel_diff_akima = np.abs((y_cubic - y_akima) / np.where(y_cubic != 0, y_cubic, np.inf))
    rel_diff_linear[np.isnan(rel_diff_linear)] = 0 
    rel_diff_quadratic[np.isnan(rel_diff_quadratic)] = 0
    rel_diff_akima[np.isnan(rel_diff_akima)] = 0


    if plot_axis is None:
      plot_axis = plt
      # Plot differences
      line_linear = plot_axis.plot(x_fine, diff_linear, label='Cubic - Linear')
      plot_axis.plot(x_fine, rel_diff_linear, linestyle='--', label='Cubic - Linear (Relative)', color=line_linear[0].get_color())

      line_quadratic = plot_axis.plot(x_fine, diff_quadratic, label='Cubic - Quadratic')
      plot_axis.plot(x_fine, rel_diff_quadratic, linestyle='--', label='Cubic - Quadratic (Relative)', color=line_quadratic[0].get_color())

      line_akima = plot_axis.plot(x_fine, diff_akima, label='Cubic - Akima')
      plot_axis.plot(x_fine, rel_diff_akima, linestyle='--', label='Cubic - Akima (Relative)', color=line_akima[0].get_color())


      plot_axis.legend()
      plot_axis.title('Differences Between Cubic Spline and Other Methods')
      plot_axis.xlabel('t(M)')
      plot_axis.ylabel('Absolute Difference')
      plot_axis.yscale('log')
      plot_axis.show()
    else:
      line_linear = plot_axis.plot(x_fine, diff_linear, label='Cubic - Linear')
      plot_axis.plot(x_fine, rel_diff_linear, linestyle='--', label='Cubic - Linear (Relative)', color=line_linear[0].get_color())

      line_quadratic = plot_axis.plot(x_fine, diff_quadratic, label='Cubic - Quadratic')
      plot_axis.plot(x_fine, rel_diff_quadratic, linestyle='--', label='Cubic - Quadratic (Relative)', color=line_quadratic[0].get_color())

      line_akima = plot_axis.plot(x_fine, diff_akima, label='Cubic - Akima')
      plot_axis.plot(x_fine, rel_diff_akima, linestyle='--', label='Cubic - Akima (Relative)', color=line_akima[0].get_color())


      plot_axis.legend()
      plot_axis.set_title('Differences Between Cubic Spline and Other Methods')
      plot_axis.set_xlabel('t(M)')
      plot_axis.set_ylabel('Absolute Difference')
      plot_axis.set_yscale('log')

    # Print maximum differences
    print(f"Max difference (Cubic - Linear): {diff_linear.max()}")
    print(f"Max difference (Cubic - Quadratic): {diff_quadratic.max()}")
    print(f"Max difference (Cubic - Akima): {diff_akima.max()}")
    print(f"Max relative difference (Cubic - Linear): {rel_diff_linear.max()}")
    print(f"Max relative difference (Cubic - Quadratic): {rel_diff_quadratic.max()}")
    print(f"Max relative difference (Cubic - Akima): {rel_diff_akima.max()}")



def find_cutoff_chebyshev(coeffs,window=100,threshold=0.005, consecutive=5, plot_things=False):
    n = len(an)
    # Function to calculate the rate of decay
    def calculate_decay_rate(coeffs, window):
        log_coeffs = np.log(np.abs(coeffs))
        return np.array([np.mean(log_coeffs[i:i+window] - log_coeffs[i-1:i+window-1]) 
                        for i in range(1, len(coeffs)-window+1)])


    # Function to find the cutoff index
    def find_cutoff(decay_rate, threshold, consecutive):
        for i in range(len(decay_rate) - consecutive + 1):
            if all(abs(rate) < threshold for rate in decay_rate[i:i+consecutive]):
                return i
        return len(decay_rate)

    decay_rate = calculate_decay_rate(coeffs,window)
    cutoff = find_cutoff(decay_rate, threshold, consecutive)
    # print(f"Suggested cutoff index: {cutoff}")

    if plot_things:
      # Plot the coefficients on a log scale
      plt.figure(figsize=(10, 6))
      plt.semilogy(range(n), coeffs, '.')
      plt.xlabel('Coefficient index')
      plt.ylabel('Absolute value of coefficient')
      plt.title('Chebyshev Coef')
      plt.grid(True)


      # Calculate and plot the decay rate

      plt.figure(figsize=(10, 6))
      plt.plot(range(len(decay_rate)), decay_rate, '.')
      plt.axhline(y=0, color='k', linestyle='--')
      plt.xlabel('Coefficient index')
      plt.ylabel('Decay rate')
      plt.title('Decay Rate of Chebyshev Coefficients')
      plt.grid(True)


      plt.show()
    return cutoff


In [ ]:
x_min,x_max = -2,1
pts = np.linspace(x_min,x_max,1000)
y = np.sin(pts)

N = 100

c_fit_chebyshev_dctII = fit_chebyshev_dctII(np.sin,N,x_min,x_max)

plt.plot(pts,evaluate_chebyshevII(c_fit_chebyshev_dctII,pts)-y,label='fit_chebyshev_dctII')

plt.legend()
plt.show()

In [ ]:
def fun1(x,amp):
    return amp*(np.sin(x)+np.cos(np.pi*x)) 
    # return amp*(np.sin(x)+np.cos(np.pi*x)) + 1

N = 30
amp = 1e-6

x_min,x_max = -1,1
pts = np.linspace(x_min,x_max,1000)
y = fun1(pts,amp)

func = lambda x:fun1(x,amp)
C = fit_chebyshev_dctII(func,N,-1,1)

plt.scatter(np.arange(len(C)),np.log10(np.abs(C)))
plt.show()


# Chebyshev fitting

In [ ]:
data_path = Path("./BondiCceR0250.h5")
data_path = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev5/BondiCceR0258/BondiCceR0258.h5")

In [ ]:
data_dict = horizon_to_pandas(data_path)
print(list(data_dict.keys()))

In [ ]:
N = 1000
t_min,t_max = 2500,5500
t = np.linspace(t_min,t_max,N)

func = CubicSpline(data_dict['J.dat']['t(M)'], data_dict['J.dat']['Re(2,2)'])

an = fit_chebyshev_dctII(func,N,t_min,t_max )

difference = np.abs(evaluate_chebyshevII(an[:200],t)-func(t))

# Create a figure and subplots
plt.close()
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
plt.subplots_adjust(hspace=0.3)  # Add space between rows

# Plot the original function and the reconstructed function on the first subplot
axes[0, 0].plot(data_dict['J.dat']['t(M)'], data_dict['J.dat']['Re(2,2)'], label='Original function')
axes[0, 0].plot(t, func(t), label='Fitted part')
axes[0, 0].legend()

# Plot the difference on the second subplot
axes[0, 1].semilogy(t, difference, label='Difference')
axes[0, 1].set_title('Difference (Original - Reconstructed)')
axes[0, 1].legend()

# Plot the scatter plots on the third row
ax_scatter = axes[1, 0]
ax_scatter.scatter(np.arange(len(an)), np.abs(an), label='an', marker='.')
ax_scatter.set_yscale('log')
ax_scatter.set_title('Fourier Coefficients')
ax_scatter.legend()

plot_interpolation_error(data_dict['J.dat']['t(M)'], data_dict['J.dat']['Re(2,2)'],x_min=t_min,x_max=t_max,N_interp=N,plot_axis=axes[1,1])

plt.tight_layout()
# Show the plots
plt.show()

In [ ]:

find_cutoff_chebyshev(an,window=100,threshold=0.01, consecutive=5, plot_things=False)

val = []
# for window,threshold,consecutive in itertools.product(np.arange(25,200,5),[0.01,0.005],[3,5,7]):
#   val.append(find_cutoff_chebyshev(an,window,threshold, consecutive, plot_things=False))
for window,threshold,consecutive in itertools.product(np.arange(25,300,5),[0.005],[7]):
  val.append(find_cutoff_chebyshev(an,window,threshold, consecutive, plot_things=False))

plt.plot(val)

In [ ]:
pts = np.linspace(2500,6000,1000)
y = func(pts)
N = 250

c_fit_chebyshev_dctII = fit_chebyshev_dctII(func,N,2500,6000)

# plt.plot(pts,evaluate_chebyshevII(c_fit_chebyshev_dctII,pts),label='fit_chebyshev_dctII')
plt.plot(pts,evaluate_chebyshevII(c_fit_chebyshev_dctII,pts)-y,label='fit_chebyshev_dctII')

plt.legend()
plt.show()

# Rough

In [ ]:
save_base_path = Path(f"/home/hchaudha/notes/spec_accuracy/world_tube_data_plot/high_accuracy_lev5_R0258")
save_base_path = Path(f"/home/hchaudha/notes/spec_accuracy/world_tube_data_plot/del")
if not save_base_path.exists():
  save_base_path.mkdir()

In [ ]:
max_plots_per_var = 5

fig, ax = plt.subplots()
t_min = 1200
t_max = 10000
for var in data_dict.keys():
    num_stop = 0
    for col in data_dict[var]:
        if 't(M)' == col:
          continue
        if 'Im' in col:
          continue
        num_stop += 1
        if num_stop > max_plots_per_var:
          break
        
        start_time = time.time()  # Start timing

        filter = (data_dict[var]['t(M)']<t_max) & (data_dict[var]['t(M)']>t_min)

        ax.clear()  # Clear the axes instead of closing the figure
        ax.plot(data_dict[var]['t(M)'][filter], data_dict[var][col][filter], label=f"{var}_{col}_t={t_min},{t_max}")
        ax.legend()
        plt.tight_layout()
        plt.savefig(save_base_path/f"{var}_{col}_t={t_min},{t_max}.png")
        
        end_time = time.time()  # End timing
        execution_time = end_time - start_time  # Calculate execution time
        
        print(f"{var}_{col} saved. Time taken: {execution_time:.4f} seconds")

plt.close(fig)  # Close the figure at the end

In [ ]:
filter = (data_dict[var]['t(M)']<t_max) & (data_dict[var]['t(M)']>t_min)
plt.plot(data_dict[var]['t(M)'][filter], data_dict[var]['Re(10,10)'][filter])

In [ ]:
generate_columns(579)